In [10]:
# Import dependencies and connect to the XRPL
import xrpl
import nest_asyncio
nest_asyncio.apply()
test_url = 'https://s.altnet.rippletest.net:51234'
client = xrpl.clients.JsonRpcClient(test_url)

In [11]:
# Generating credentials for Testnet Faucet
faucet_url = "https://faucet.altnet.rippletest.net/accounts"
print("Getting 2 new accounts from the Testnet faucet")
from xrpl.wallet import generate_faucet_wallet
cold_wallet = generate_faucet_wallet(client, debug = True)
hot_wallet = generate_faucet_wallet(client, debug=True)

Getting 2 new accounts from the Testnet faucet
Attempting to fund address rstsTDN8bZRiA4bhD5P8LiaXWzNFzM5cgw


C:\Users\rlkee\anaconda3\lib\site-packages\base58\__init__.py:72: RuntimeWarning: coroutine 'generate_faucet_wallet' was never awaited
  invmap = {char: index for index, char in enumerate(alphabet)}


Faucet fund successful.
Attempting to fund address rfwzaPeyPqG2vZuvEipGKCPYWsYW5D4uWq
Faucet fund successful.


In [14]:
# Stand-alone code sample for the "issue a token" tutorial:
# https://xrpl.org/issue-a-fungible-token.html
# License: https://github.com/XRPLF/xrpl-dev-portal/blob/master/LICENSE

# Connect ----------------------------------------------------------------------
import xrpl
testnet_url = "https://s.altnet.rippletest.net:51234"
client = xrpl.clients.JsonRpcClient(testnet_url)


# Get credentials from the Testnet Faucet --------------------------------------
# For production, instead create a Wallet instance
faucet_url = "https://faucet.altnet.rippletest.net/accounts"
print("Getting 2 new accounts from the Testnet faucet...")
from xrpl.wallet import generate_faucet_wallet
cold_wallet = generate_faucet_wallet(client, debug=True)
hot_wallet = generate_faucet_wallet(client, debug=True)


# Configure issuer (cold address) settings -------------------------------------
cold_settings_tx = xrpl.models.transactions.AccountSet(
    account=cold_wallet.classic_address,
    transfer_rate=0,
    tick_size=5,
    domain=bytes.hex("example.com".encode("ASCII")),
    set_flag=xrpl.models.transactions.AccountSetFlag.ASF_DEFAULT_RIPPLE,
)
cst_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=cold_settings_tx,
    wallet=cold_wallet,
    client=client,
)
print("Sending cold address AccountSet transaction...")
response = xrpl.transaction.send_reliable_submission(cst_prepared, client)
print(response)


# Configure hot address settings -----------------------------------------------
hot_settings_tx = xrpl.models.transactions.AccountSet(
    account=hot_wallet.classic_address,
    set_flag=xrpl.models.transactions.AccountSetFlag.ASF_REQUIRE_AUTH,
)
hst_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=hot_settings_tx,
    wallet=hot_wallet,
    client=client,
)
print("Sending hot address AccountSet transaction...")
response = xrpl.transaction.send_reliable_submission(hst_prepared, client)
print(response)


# Create trust line from hot to cold address -----------------------------------
currency_code = "XDA"
trust_set_tx = xrpl.models.transactions.TrustSet(
    account=hot_wallet.classic_address,
    limit_amount=xrpl.models.amounts.issued_currency_amount.IssuedCurrencyAmount(
        currency=currency_code,
        issuer=cold_wallet.classic_address,
        value="9000000000", # Large limit, arbitrarily chosen
    )
)
ts_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=trust_set_tx,
    wallet=hot_wallet,
    client=client,
)
print("Creating trust line from hot address to issuer...")
response = xrpl.transaction.send_reliable_submission(ts_prepared, client)
print(response)


# Send token -------------------------------------------------------------------
issue_quantity = "500000"
send_token_tx = xrpl.models.transactions.Payment(
    account=cold_wallet.classic_address,
    destination=hot_wallet.classic_address,
    amount=xrpl.models.amounts.issued_currency_amount.IssuedCurrencyAmount(
        currency=currency_code,
        issuer=cold_wallet.classic_address,
        value=issue_quantity
    )
)
pay_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=send_token_tx,
    wallet=cold_wallet,
    client=client,
)
print(f"Sending {issue_quantity} {currency_code} to {hot_wallet.classic_address}...")
response = xrpl.transaction.send_reliable_submission(pay_prepared, client)
print(response)


# Check balances ---------------------------------------------------------------
print("Getting hot address balances...")
response = client.request(xrpl.models.requests.AccountLines(
    account=hot_wallet.classic_address,
    ledger_index="validated",
))
print(response)

print("Getting cold address balances...")
response = client.request(xrpl.models.requests.GatewayBalances(
    account=cold_wallet.classic_address,
    ledger_index="validated",
    hotwallet=[hot_wallet.classic_address]
))
print(response)

Getting 2 new accounts from the Testnet faucet...
Attempting to fund address rPFEMbUQrBQccJk4gh7iCCFLRNzRtPpctU
Faucet fund successful.
Attempting to fund address rMmX6S2vdiMi73gaX7tPAtxvivimQbvnC2
Faucet fund successful.
Sending cold address AccountSet transaction...
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'rPFEMbUQrBQccJk4gh7iCCFLRNzRtPpctU', 'Domain': '6578616D706C652E636F6D', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 29787209, 'Sequence': 29787185, 'SetFlag': 8, 'SigningPubKey': 'ED6CE457C65962B9ADB387E8F0EEE706A5EDC3FCB2FF7031B524FEE606175ACCB8', 'TickSize': 5, 'TransactionType': 'AccountSet', 'TransferRate': 0, 'TxnSignature': '039723228B64631C1F0D141530A20E657D620EFF67A6A4F9CECBFF867D5F2FBF41BB9AAFB4242E1DEF2367473D2739C158300562414DE2523EFFBB5E8E5F470E', 'date': 712117461, 'hash': '230B27F0905861C1342BB1F05C963E135BF0255CB1AC689CB71D9DAEA284FB36', 'inLedger': 29787191, 'ledger_index': 29787191, 'meta': {'AffectedNodes': [{'ModifiedNode': {'F

In [36]:
response

Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'account': 'rPFEMbUQrBQccJk4gh7iCCFLRNzRtPpctU', 'balances': {'rMmX6S2vdiMi73gaX7tPAtxvivimQbvnC2': [{'currency': 'XDA', 'value': '500000'}]}, 'ledger_hash': '694C4F489C933F79DA65BC9535F05450FDA8CE5C573CD5216A80FF28A2FF7825', 'ledger_index': 29787200, 'validated': True}, id=None, type=<ResponseType.RESPONSE: 'response'>)